In [1]:
%matplotlib
import numpy as np
from scipy.optimize import root, brentq
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D, axes3d
import math

Using matplotlib backend: TkAgg


In [24]:
w = 0.4
alpha = 0.3
sigma = 0.4
mu = 0.2
r = 0.05
theta = (mu - r)/sigma
b = 0.3
T = 1
x0 = 1
def theta_tilda(x):
    if x >= x0:
        return a * (x - x0)
    if (1 - w) * x0 <= x < x0:
        return x - x0
    if x < (1 - w) * x0:
        return -w * x0
a = w + alpha * (1 - w)
z0 = theta_tilda(x0 / math.exp(-r - theta ** 2))

In [3]:
def inverse_xi(x):
    return (- (r + theta ** 2 / 2) * T - math.log(x)) / theta / math.sqrt(T)


def zero_moment(downbdd, upbdd):
    if upbdd == 'infty':
        if downbdd > 0:
            return (1 + math.erf(inverse_xi(downbdd) / math.sqrt(2))) / 2
        else:
            return 1
    elif downbdd < upbdd:
        if downbdd > 0:
            return (math.erf(inverse_xi(downbdd) / math.sqrt(2)) - math.erf(inverse_xi(upbdd) / math.sqrt(2))) / 2
        elif upbdd > 0:
            return (1 - math.erf(inverse_xi(upbdd) / math.sqrt(2))) / 2
        else:
            return 0
    else:
        return 0


def first_moment(downbdd, upbdd):
    if upbdd == 'infty':
        if downbdd > 0:
            return math.exp(-r * T) * \
                   (1 + math.erf((inverse_xi(downbdd) + math.sqrt(T) * theta) / math.sqrt(2))) / 2
        else:
            return math.exp(-r * T)
    if downbdd < upbdd:
        if downbdd > 0:
            return math.exp(-r * T) * (math.erf((inverse_xi(downbdd) + math.sqrt(T) * theta) / math.sqrt(2)) -
                    math.erf((inverse_xi(upbdd) + math.sqrt(T) * theta) / math.sqrt(2))) / 2
        elif upbdd > 0:
            return math.exp(-r * T) * \
                   (1 - math.erf((inverse_xi(upbdd) + math.sqrt(T) * theta) / math.sqrt(2))) / 2
        else:
            return 0
    else:
        return 0


def second_moment(downbdd, upbdd):
    if upbdd == 'infty':
        if downbdd > 0:
            return math.exp((- 2 * r + theta ** 2) * T) * \
                   (1 + math.erf((inverse_xi(downbdd) + 2 * math.sqrt(T) * theta) / math.sqrt(2))) / 2
        else:
            return math.exp((- 2 * r + theta ** 2) * T)
    if downbdd < upbdd:
        if downbdd > 0:
            return math.exp((- 2 * r + theta ** 2) * T) * (math.erf((inverse_xi(downbdd) + 2 * math.sqrt(T) * theta)
                    / math.sqrt(2)) - math.erf((inverse_xi(upbdd) + 2 * math.sqrt(T) * theta) / math.sqrt(2))) / 2
        elif upbdd > 0:
            return math.exp((- 2 * r + theta ** 2) * T) * \
                   (1 - math.erf((inverse_xi(upbdd) + 2 * math.sqrt(T) * theta) / math.sqrt(2))) / 2
        else:
            return 0
    else:
        return 0

In [4]:
def subsituation(lda, muu):
    if muu == 0:
        if lda / 2 / (w + alpha * (1 - w)) > 0:
            if - lda ** 2 / 4 < w ** 2 * x0 ** 2 + lda * w * x0:
                return [['five', 0, 'infty']]
            else:
                return [['two', 0, 'infty']]
        else:
            if 0 < w ** 2 * x0 ** 2 + lda * w * x0:
                return [['four', 0, 'infty']]
            else:
                return [['two', 0, 'infty']]
    else:
        cutting1 = - lda * (w + alpha * (1 - w)) / muu
        cutting2 = - 2 * lda * (w + alpha * (1 - w)) / (muu * (1 + w + alpha * (1 - w)))
        cutting5_b = (w ** 2 * x0 ** 2 + lda * w * x0) / muu / x0 / (b - 1)
        cutting5_1_w = - (w * x0 ** 2 + lda * x0) / muu / x0
        cutting6 = (- 2 * w * x0 - lda) / muu
        cutting7 = - lda / muu
        delta3_b = 2 * lda * muu * (w + alpha * (1 - w)) ** 2 + 2 * muu * x0 - 4 * muu * b * x0 * \
                    (w + alpha * (1 - w)) ** 2 - 4 * muu ** 2 * (2 * lda ** 2 * (w + alpha * (1 - w)) ** 2 +
                    4 * (w + alpha * (1 - w)) ** 2 * (w ** 2 * x0 ** 2 + lda * w * x0))
        delta3_1_w = 2 * lda * muu * (w + alpha * (1 - w)) ** 2 + 2 * muu * x0 - 4 * muu * (1 - w) * x0 * \
                    (w + alpha * (1 - w)) ** 2 - 4 * muu ** 2 * (2 * lda ** 2 * (w + alpha * (1 - w)) ** 2 +
                    4 * (w + alpha * (1 - w)) ** 2 * (w ** 2 * x0 ** 2 + lda * w * x0))
        delta4_b = (2 * muu * lda - muu * b * x0 + 4 * muu * x0) ** 2 - 4 * muu ** 2 * \
                   (lda ** 2 + 4 * (w ** 2 * x0 ** 2 + lda * w * x0))
        delta4_1_w = (2 * muu * lda - muu * (1 - w) * x0 + 4 * muu * x0) ** 2 - 4 * muu ** 2 * \
                    (lda ** 2 + 4 * (w ** 2 * x0 ** 2 + lda * w * x0))
        if muu > 0:
            if delta3_1_w <= 0:
                e111 = [['five', cutting1, cutting6]]
            else:
                cutting3_1_w_low = (-(2 * muu * lda * (w + alpha * (1 - w)) + 2 * muu * x0 - 4 * muu * b * x0 *
                                      (w + alpha * (1 - w)) ** 2) - math.sqrt(delta3_1_w)) / 2 / muu ** 2
                cutting3_1_w_high = (-(2 * muu * lda * (w + alpha * (1 - w)) + 2 * muu * x0 - 4 * muu * b * x0 *
                                       (w + alpha * (1 - w)) ** 2) + math.sqrt(delta3_1_w)) / 2 / muu ** 2
                e111 = [['five', cutting1, min(cutting3_1_w_low, cutting6)], ['five', max(cutting1, cutting3_1_w_high), cutting6],
                        ['two', max(cutting1, cutting3_1_w_low), min(cutting3_1_w_high, cutting6)]]
            e112 = [['five', max(cutting1, cutting2, cutting6), cutting7],
                    ['three', max(cutting1, cutting6), min(cutting2, cutting7)]]
            e113 = [['five', max(cutting1, cutting7), 'infty']]
            e211 = [['two', 0, min(cutting1, cutting6)]]
            e212 = [['three', cutting6, min(cutting1, cutting7)]]
            e213 = [['four', cutting7, cutting1]]
            return e111 + e112 + e113 + e211 + e212 + e213
        else:
            if delta3_b <= 0:
                e121 = [['five', cutting6, cutting1]]
                e122 = [['five', max(cutting2, cutting7), min(cutting1, cutting6)],
                        ['three', cutting7, min(cutting1, cutting2, cutting6)]]
                e123 = [['five', 0, min(cutting1, cutting7)]]
            else:
                cutting3_b_low = (-(2 * muu * lda * (w + alpha * (1 - w)) + 2 * muu * x0 - 4 * muu * b * x0 *
                                      (w + alpha * (1 - w)) ** 2) - math.sqrt(delta3_b)) / 2 / muu ** 2
                cutting3_b_high = (-(2 * muu * lda * (w + alpha * (1 - w)) + 2 * muu * x0 - 4 * muu * b * x0 *
                                    (w + alpha * (1 - w)) ** 2) + math.sqrt(delta3_b)) / 2 / muu ** 2
                e121 = [['five', cutting6, min(cutting1, cutting3_b_low)], ['five', max(cutting3_b_high, cutting6), cutting1],
                        ['one', max(cutting6, cutting3_b_low), min(cutting1, cutting3_b_high)]]
                if delta4_b <= 0:
                    e122 = [['five', max(cutting2, cutting7), min(cutting1, cutting6)],
                            ['three', cutting7, min(cutting1, cutting2, cutting6)]]
                else:
                    cutting4_b_low = (-(2 * lda * muu - 4 * muu * b * x0 + 4 * muu * x0) - math.sqrt(delta4_b)) / 2 / muu ** 2
                    cutting4_b_high = (-(2 * lda * muu - 4 * muu * b * x0 + 4 * muu * x0) - math.sqrt(delta4_b)) / 2 / muu ** 2
                    e122 = [['five', max(cutting2, cutting7), min(cutting1, cutting3_b_low, cutting6)],
                            ['three', cutting7, min(cutting1, cutting2, cutting6, cutting4_b_low)],
                            ['one', max(cutting3_b_low, cutting4_b_low, cutting7),
                             max(cutting1, cutting3_b_high, cutting4_b_high, cutting6)]]
                e123 = [['five', 0, min(cutting1, cutting3_b_low, cutting7)], ['five', cutting3_b_high, min(cutting1, cutting7)],
                        ['one', cutting3_b_low, min(cutting1, cutting3_b_high, cutting7)]]
            e221 = [['four', cutting5_b, cutting1]]
            e222 = [['one', 0, min(cutting1, cutting5_b)]]
            return e121 + e122 + e123 + e221 + e222

In [5]:
def expectation(lda, muu):
    def five(downbdd, upbdd):
        return muu / (2 * (w + alpha * (1 - w))) * first_moment(downbdd, upbdd) + lda / 2 * zero_moment(downbdd, upbdd)

    def four(downbdd, upbdd):
        return 0

    def three(downbdd, upbdd):
        return muu / 2 * first_moment(downbdd, upbdd) + lda / 2 * zero_moment(downbdd, upbdd)

    def two(downbdd, upbdd):
        return - w * x0 * zero_moment(downbdd, upbdd)

    def one(downbdd, upbdd):
        return - w * x0 * zero_moment(downbdd, upbdd)

    sub = subsituation(lda, muu)
    expt = 0
    for item in sub:
        expt += eval(item[0])(item[1], item[2])
    return expt


def expectation2(lda, muu):
    def five(downbdd, upbdd):
        return (lda / (2 * (w + alpha * (1 - w))) + x0) * first_moment(downbdd, upbdd) + \
               muu / (2 * (w + alpha * (1 - w)) ** 2) * second_moment(downbdd, upbdd)

    def four(downbdd, upbdd):
        return x0 * first_moment(downbdd, upbdd)

    def three(downbdd, upbdd):
        return (lda / 2 + x0) * first_moment(downbdd, upbdd) + muu / 2 * second_moment(downbdd, upbdd)

    def two(downbdd, upbdd):
        return (1 - w) * x0 * zero_moment(downbdd, upbdd)

    def one(downbdd, upbdd):
        return b * x0 * zero_moment(downbdd, upbdd)

    sub = subsituation(lda, muu)
    expt = 0
    for item in sub:
        expt += eval(item[0])(item[1], item[2])
    return expt


def square_of_l2_norm(lda, muu):
        def five(downbdd, upbdd):
            return muu ** 2 / 4 / (w + alpha * (1 - w)) ** 2 * second_moment(downbdd, upbdd) + \
                   lda * muu / (w + alpha * (1 - w)) * first_moment(downbdd, upbdd) / 2 + \
                   lda ** 2 / 4 * zero_moment(downbdd, upbdd)

        def four(downbdd, upbdd):
            return 0

        def three(downbdd, upbdd):
            return muu ** 2 / 4 * second_moment(downbdd, upbdd) + lda * muu * first_moment(downbdd, upbdd) / 2 + \
                   lda ** 2 / 4 * zero_moment(downbdd, upbdd)

        def two(downbdd, upbdd):
            return w ** 2 * x0 ** 2 * zero_moment(downbdd, upbdd)

        def one(downbdd, upbdd):
            return w ** 2 * x0 ** 2 * zero_moment(downbdd, upbdd)

        sub = subsituation(lda, muu)
        expt = 0
        for item in sub:
            expt += eval(item[0])(item[1], item[2])
        return expt

In [4]:
def subsituation(lda, muu):
    if muu > 0:
        return [['one', (2 * a ** 2 * (b - 1) * x0 - lda * a) / muu, 'infty'],
                ['two', 0, (2 * a ** 2 * (b - 1) * x0 - lda * a) / muu]]
    elif muu < 0:
        return [['one', 0, (2 * a ** 2 * (b - 1) * x0 - lda * a) / muu],
                ['two', (2 * a ** 2 * (b - 1) * x0 - lda * a) / muu, 'infty']]


def expectation(lda, muu):
    if muu == 0:
        return a * (max(x0 + lda / 2 / a, b * x0) - x0)

    def one(downbdd, upbdd):
        return (x0 + lda / 2 / a) * zero_moment(downbdd, upbdd) + muu / 2 / a ** 2 * first_moment(downbdd, upbdd)

    def two(downbdd, upbdd):
        return b * x0 * zero_moment(downbdd, upbdd)

    sub = subsituation(lda, muu)
    expt = 0
    for item in sub:
        expt += eval(item[0])(item[1], item[2])
        
        # print(eval(item[0])(item[1], item[2]), b * x0 * zero_moment(item[1], item[2]))
        # print('item1=', item[1], 'item2=', item[2])
        # print('expt=', expt)
        
    return a * (expt - x0)


def expectation2(lda, muu):
    if muu == 0:
        return max(x0 + lda / 2 / a, b * x0) * first_moment(0, 'infty')

    def one(downbdd, upbdd):
        return (x0 + lda / 2 / a) * first_moment(downbdd, upbdd) + muu / 2 / a ** 2 * second_moment(downbdd, upbdd)

    def two(downbdd, upbdd):
        return b * x0 * zero_moment(downbdd, upbdd)

    sub = subsituation(lda, muu)
    expt = 0
    for item in sub:
        expt += eval(item[0])(item[1], item[2])
    return expt


def square_of_l2_norm(lda, muu):
    if muu == 0:
        return a ** 2 * (max(x0 + lda / 2 / a, b * x0) - x0) ** 2

    def one(downbdd, upbdd):
        return (lda / 2 / a) ** 2 * zero_moment(downbdd, upbdd) + lda / a * muu / 2 / a ** 2 * first_moment(downbdd, upbdd) + \
                muu ** 2 / 4 / a ** 4 * second_moment(downbdd, upbdd)

    def two(downbdd, upbdd):
        return (b - 1) ** 2 * x0 ** 2 * zero_moment(downbdd, upbdd)

    sub = subsituation(lda, muu)
    expt = 0
    for item in sub:
        expt += eval(item[0])(item[1], item[2])
    return a ** 2 * expt

In [19]:
def mv(z):
    def constraint(x):
        return expectation(x[0], x[1]) - z, expectation2(x[0], x[1]) - x0

    sol = root(constraint, [0, 0])
    lda, muu = sol.x
    success = sol.success
    
    print('lda=', lda, 'muu=', muu)
    print('expectation=', expectation(lda, muu), 'expectation2=', expectation2(lda, muu))
    print('w=', w, 'sigma=', sigma, 'z=', z)
    print('success=', success)

    return square_of_l2_norm(lda, muu) - z ** 2

In [25]:
# theta = 0.5
# z0 = a * (x0 / math.exp(-r + theta ** 2) - x0)
mv(z0)

lda= 0.10663057511783293 muu= 0.00015180560687329822
expectation= 0.05343977200724583 expectation2= 1.0389041885356303
w= 0.4 sigma= 0.4 z= 0.12180325663420564
success= False


-0.011980221754668018

In [10]:
fig, ax = plt.subplots()
ax.plot(np.linspace(-0.1, 1), [mv(z) for z in np.linspace(-0.1, 1)])

lda= -0.9810706913466151 muu= 0.6796989821696761
expectation= -0.09999999999999978 expectation2= 1.0
w= 0.5 sigma= 0.4 z= -0.1
success= True
lda= -0.8140931412923201 muu= 0.5704893148454145
expectation= -0.07755102040743223 expectation2= 1.0000000000004168
w= 0.5 sigma= 0.4 z= -0.07755102040816327
success= True
lda= -0.6471402679274912 muu= 0.4612417939545862
expectation= -0.055102040816362725 expectation2= 0.9999999999999784
w= 0.5 sigma= 0.4 z= -0.05510204081632653
success= True
lda= -0.48024352719188546 muu= 0.3519346790230536
expectation= -0.032653061224566245 expectation2= 0.9999999999999514
w= 0.5 sigma= 0.4 z= -0.0326530612244898
success= True
lda= -0.3135027349699256 muu= 0.24253638929391505
expectation= -0.010204081632943316 expectation2= 0.9999999999997988
w= 0.5 sigma= 0.4 z= -0.010204081632653059
success= True
lda= -0.14725949255098697 muu= 0.13307173654477591
expectation= 0.0122448979591838 expectation2= 1.0
w= 0.5 sigma= 0.4 z= 0.012244897959183682
success= True
lda= 0.06

lda= 0.07451341228674797 muu= 0.004514425689546621
expectation= 0.04055997887461912 expectation2= 1.0113160258803595
w= 0.5 sigma= 0.4 z= 0.05714285714285716
success= False
lda= 0.0936814594826831 muu= 0.0006030927389812997
expectation= 0.04728202170981981 expectation2= 1.0205208592612416
w= 0.5 sigma= 0.4 z= 0.07959183673469389
success= False
lda= 0.10654393444553426 muu= 0.0011161987057874503
expectation= 0.05408870649394762 expectation2= 1.0305649268689783
w= 0.5 sigma= 0.4 z= 0.10204081632653061
success= False
lda= 0.12349764880154687 muu= 8.827633588025377e-05
expectation= 0.06181341751475529 expectation2= 1.0417032998991005
w= 0.5 sigma= 0.4 z= 0.12448979591836737
success= False
lda= 0.13315985004717257 muu= 7.289908581208249e-05
expectation= 0.06663326637392755 expectation2= 1.0487543244801734
w= 0.5 sigma= 0.4 z= 0.1469387755102041
success= False


lda= 0.15007998374193304 muu= 0.003623280663227187
expectation= 0.0776912004710328 expectation2= 1.0655108801503368
w= 0.5 sigma= 0.4 z= 0.16938775510204082
success= False
lda= 0.16215364381713832 muu= 0.00010986105903333605
expectation= 0.0811572088869994 expectation2= 1.0700150721922737
w= 0.5 sigma= 0.4 z= 0.19183673469387755
success= False
lda= 0.17629117374974995 muu= 4.232940455076655e-05
expectation= 0.0881765599326675 expectation2= 1.080276481402886
w= 0.5 sigma= 0.4 z= 0.21428571428571433
success= False


lda= 0.18790224752648454 muu= 0.00043881954158119874
expectation= 0.09427221457862531 expectation2= 1.0892611511943822
w= 0.5 sigma= 0.4 z= 0.23673469387755106
success= False
lda= 0.20283405207802144 muu= 0.0011219008302227244
expectation= 0.10223793763984121 expectation2= 1.101028872561765
w= 0.5 sigma= 0.4 z= 0.25918367346938775
success= False
lda= 0.21538735796746467 muu= 0.0005391988360447101
expectation= 0.10808821882873408 expectation2= 1.109496134544232
w= 0.5 sigma= 0.4 z= 0.28163265306122454
success= False


lda= 0.22581936551092321 muu= 0.0028807458409075516
expectation= 0.11501756753113801 expectation2= 1.1200153442068335
w= 0.5 sigma= 0.4 z= 0.3040816326530612
success= False
lda= 0.23904784560292522 muu= 0.005341632613228899
expectation= 0.12343247519882874 expectation2= 1.1327278607378217
w= 0.5 sigma= 0.4 z= 0.326530612244898
success= False
lda=

...

 0.36540714334195823 muu= 0.0009536390154366576
expectation= 0.18340136358777545 expectation2= 1.2197786501098393
w= 0.5 sigma= 0.4 z= 0.5285714285714287
success= False
lda= 0.37670483852327263 muu= 0.0022165456933603135
expectation= 0.18997429886492453 expectation2= 1.229601873445746
w= 0.5 sigma= 0.4 z= 0.5510204081632654
success= False
lda= 0.3917308286370074 muu= 0.0013475304479582946
expectation= 0.19685142248197182 expectation2= 1.2395255546838202
w= 0.5 sigma= 0.4 z= 0.5734693877551021
success= False
lda= 0.4067428630556461 muu= 0.00048747512970266854
expectation= 0.20372812436404267 expectation2= 1.2494500674307054
w= 0.5 sigma= 0.4 z= 0.5959183673469388
success= False


lda= 0.4187558746212412 muu= 9.680684094590802e-05
expectation= 0.20944877232262116 expectation2= 1.2577586692944702
w= 0.5 sigma= 0.4 z= 0.6183673469387756
success= False
lda= 21.820122225777332 muu= -18.336181920263215
expectation= 0.6408163265306701 expectation2= 1.0000000000003746
w= 0.5 sigma= 0.4 z= 0.6408163265306124
success= True
lda= 23.11018235014698 muu= -19.606035460504003
expectation= 0.6632653061227187 expectation2= 1.0000000000016056
w= 0.5 sigma= 0.4 z= 0.663265306122449
success= True
lda= 24.478217095429493 muu= -20.965529130346056
expectation= 0.6857142857155286 expectation2= 1.00000000000693
w= 0.5 sigma= 0.4 z= 0.6857142857142858
success= True
lda= 25.92990219228028 muu= -22.421925564464573
expectation= 0.7081632653062941 expectation2= 1.0000000000009004
w= 0.5 sigma= 0.4 z= 0.7081632653061225
success= True
lda=

 27.47140450497057 muu= -23.983170334407586
expectation= 0.7306122448979598 expectation2= 0.9999999999999999
w= 0.5 sigma= 0.4 z= 0.7306122448979593
success= True
lda= 29.10943036853819 muu= -25.657964358594484
expectation= 0.7530612244898462 expectation2= 1.0000000000000824
w= 0.5 sigma= 0.4 z= 0.7530612244897961
success= True
lda= 30.851279168924286 muu= -27.455844749140272
expectation= 0.7755102040816475 expectation2= 1.000000000000023
w= 0.5 sigma= 0.4 z= 0.7755102040816327
success= True
lda= 32.70490277781965 muu= -29.38727515376095
expectation= 0.7979591836734599 expectation2= 0.9999999999999816
w= 0.5 sigma= 0.4 z= 0.7979591836734695
success= True
lda= 34.67897152322689 muu= -31.46374678429301
expectation= 0.8204081632649555 expectation2= 0.9999999999995115
w= 0.5 sigma= 0.4 z= 0.8204081632653062
success= True
lda=

 36.78294745953837 muu= -33.69789148033588
expectation= 0.8428571428571268 expectation2= 0.9999999999999795
w= 0.5 sigma= 0.4 z= 0.842857142857143
success= True
lda= 39.02716579278011 muu= -36.103608333476885
expectation= 0.8653061224490577 expectation2= 1.0000000000001157
w= 0.5 sigma= 0.4 z= 0.8653061224489798
success= True
lda= 41.42292539397981 muu= -38.69620556166149
expectation= 0.8877551026492018 expectation2= 1.0000000010119952
w= 0.5 sigma= 0.4 z= 0.8877551020408164
success= True
lda= 43.98258964956813 muu= -41.492559834780764
expectation= 0.9102040816324555 expectation2= 0.9999999999996779
w= 0.5 sigma= 0.4 z= 0.9102040816326532
success= True
lda= 46.71969832088186 muu= -44.511294510476716
expectation= 0.9326530612281163 expectation2= 1.0000000000060274
w= 0.5 sigma= 0.4 z= 0.93265306122449
success= True
lda= 49.6490926378839 muu= -47.77298052515608
expectation= 0.9551020408162278 expectation2= 0.9999999999998497
w= 0.5 sigma= 0.4 z= 0.9551020408163265
success= True
lda=

 52.78705424763399 muu= -51.30036151492261
expectation= 0.9775510203526302 expectation2= 0.9999999999075022
w= 0.5 sigma= 0.4 z= 0.9775510204081633
success= True
lda= 56.1514602017767 muu= -55.11860707442964
expectation= 1.00000000001432 expectation2= 1.0000000000238527
w= 0.5 sigma= 0.4 z= 1.0
success= True


In [205]:
fig, ax = plt.subplots()
ax.plot([expectation(-1, i) for i in np.linspace(-50, 50, 1000)], 
        [square_of_l2_norm(-1, i) for i in np.linspace(-50, 50, 1000)])

In [199]:
# theta = 0.5
# z0 = a * (x0 / math.exp(-r + theta ** 2) - x0)
for i in np.linspace(-10, 10):
    print(expectation(1, i), square_of_l2_norm(1, i))

0.08778620108835647 0.008005014210152062
0.08799988844059171 0.008080297286365124
0.08824661471456968 0.008167595377899664
0.08853279983946856 0.00826931675405993
0.08886639800579177 0.008388463125911001
0.0892573410978595 0.00852880647863002
0.0897181350994178 0.00869512880754962
0.09026467182040983 0.008893551206309311
0.09091734807980728 0.009131991800174032
0.09170263084806636 0.00942081256666951
0.09265528051274971 0.009773748154584351
0.09382156409367903 0.0102092643021462
0.09526398951947214 0.010752585635160615
0.09706843330383494 0.011438793237073079
0.09935513674996536 0.012317681719083892
0.10229614903510446 0.013461606963804173
0.10614389814551321 0.014978614385234948
0.11127977036635438 0.017035316340939723
0.11830044128309615 0.019898752295074477
0.12817966461705613 0.024017680467628533
0.1425918898340697 0.030192661576569142
0.16461480775104056 0.03996803888319999
0.20042415357514023 0.05666249441711901
0.26397935414108464 0.08865697683851861
0.3916532129922192 0.1638762

In [21]:
fig, ax = plt.subplots()
points = np.array([[i, mv(i)[0]] for i in np.linspace(z0-0.5, z0+0.5) if mv(i)[1]]).T.reshape(2, -1)
ax.plot(points[0], points[1])

In [25]:
math.exp(-0.2)*1.1

0.9006038283857801

In [76]:
fig, ax = plt.subplots(3, 3, sharex=True, sharey=True)
for i in enumerate([0.3, 0.4, 0.8]):
    for j in enumerate([10, 2, 1.25]):
        w = i[1]
        # theta = j[1]
        sigma = j[1]
        theta = (mu - r)/sigma
        a = w + alpha * (1 - w)
        z0 = a * (x0 / math.exp(-r + theta ** 2) - x0)
        points = np.array([[z, mv(z)[0]] for z in np.linspace(z0-0.5, z0+0.5) if mv(z)[1]]).T.reshape(2, -1)
        ax[i[0], j[0]].plot(points[0], points[1])
        ax[i[0], j[0]].axvline(ymax=0.8, x=z0, c='C1', linestyle='dashed')
        ax[i[0], j[0]].text(0.5, 0.95, r'$w={}, \theta={}$'.format(w, theta), fontsize=20, 
            transform=ax[i[0], j[0]].transAxes, horizontalalignment='center', verticalalignment='top')
        ax[i[0], j[0]].xaxis.set_tick_params(which='both', labelbottom=True)
for i in range(3):
    ax[i, 0].set_ylabel('variance', fontsize=20)
    ax[2, i].set_xlabel('mean', fontsize=20)

In [86]:
fig, ax = plt.subplots(3, 3, sharex=True)
for i in enumerate([0.5, 1, 2]):
    for j in enumerate([10, 2, 1.25]):
        x0 = i[1]
        sigma = j[1]
        theta = (mu - r)/sigma
        a = w + alpha * (1 - w)
        z0 = a * (x0 / math.exp(-r + theta ** 2) - x0)
        points = np.array([[z, mv(z)[0]] for z in np.linspace(z0-0.5, z0+0.5) if mv(z)[1]]).T.reshape(2, -1)
        ax[i[0], j[0]].plot(points[0], points[1])
        ax[i[0], j[0]].axvline(ymax=0.8, x=z0, c='C1', linestyle='dashed')
        ax[i[0], j[0]].text(0.5, 0.95, r'$x_0={}, \sigma={}$'.format(x0, sigma), fontsize=20, 
            transform=ax[i[0], j[0]].transAxes, horizontalalignment='center', verticalalignment='top')
        ax[i[0], j[0]].xaxis.set_tick_params(which='both', labelbottom=True)
for i in range(3):
    ax[i, 0].set_ylabel('variance', fontsize=20)
    ax[2, i].set_xlabel('mean', fontsize=20)

In [61]:
fig, ax = plt.subplots(subplot_kw=dict(projection='3d'))
xx, yy = np.meshgrid(np.linspace(-5, 5, 20), np.linspace(-10, 10, 50))
zz = np.zeros((2, 50, 20))
for i in range(50):
    for j in range(20):
        zz[0, i, j] = expectation(xx[i, j], yy[i, j])
        zz[1, i, j] = expectation2(xx[i, j], yy[i, j]) - x0
ax.plot_surface(xx, yy, zz[0])
ax.plot_surface(xx, yy, zz[1])

some notes here:
1, mv >= 0
2, make sure that the functions are continuous in lda, muu
3, expectation >= a * (b - 1) * x0, the same as expectation2